## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-07-18-56-00 +0000,wsj,Trump Plans to Send Immigrant Detainees to Not...,https://www.wsj.com/us-news/trump-plans-to-sen...
1,2025-08-07-18-55-16 +0000,nypost,New NYC Council proposal will tie up businesse...,https://nypost.com/2025/08/07/us-news/new-nyc-...
2,2025-08-07-18-54-10 +0000,bbc,Aerial footage shows Gaza in ruins,https://www.bbc.com/news/videos/cwy3nple983o?a...
3,2025-08-07-18-53-31 +0000,nyt,"Woman Working at Hospital Treated 4,500 Patien...",https://www.nytimes.com/2025/08/07/us/florida-...
4,2025-08-07-18-49-00 +0000,wsj,Trump Calls on Intel CEO to Resign Over China ...,https://www.wsj.com/tech/trump-intel-ceo-lip-b...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,78
95,tariffs,22
8,new,20
12,will,15
605,putin,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
312,2025-08-06-19-46-31 +0000,nypost,"Trump plans sitdown with Putin, Zelensky in fi...",https://nypost.com/2025/08/06/us-news/trump-pl...,138
82,2025-08-07-16-12-12 +0000,wapo,Trump celebrates new tariffs as negotiations c...,https://www.washingtonpost.com/business/2025/0...,135
161,2025-08-07-11-54-12 +0000,bbc,Trump's sweeping new tariffs take effect again...,https://www.bbc.com/news/articles/cx23jmvn5yzo...,134
117,2025-08-07-14-30-28 +0000,wapo,Trump’s new tariffs are hitting these major tr...,https://www.washingtonpost.com/business/2025/0...,132
281,2025-08-06-21-56-54 +0000,wapo,"Trump raises tariffs on India to 50 percent, c...",https://www.washingtonpost.com/business/2025/0...,126


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
312,138,2025-08-06-19-46-31 +0000,nypost,"Trump plans sitdown with Putin, Zelensky in fi...",https://nypost.com/2025/08/06/us-news/trump-pl...
228,83,2025-08-07-04-12-00 +0000,wsj,"China’s exports grew at a faster clip in July,...",https://www.wsj.com/economy/trade/chinas-expor...
15,75,2025-08-07-18-30-00 +0000,wsj,Sen. John Cornyn said the FBI will help track ...,https://www.wsj.com/politics/policy/fbi-texas-...
273,60,2025-08-06-22-24-32 +0000,nypost,GOP Sen. Tom Cotton questions new Intel CEO Li...,https://nypost.com/2025/08/06/business/gop-sen...
38,44,2025-08-07-18-00-11 +0000,bbc,"Dozens of migrants detained under 'one in, one...",https://www.bbc.com/news/articles/ce35v0zyzvlo...
250,42,2025-08-07-00-00-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...
121,38,2025-08-07-14-22-46 +0000,nypost,Gunman charged in deadly Israeli Embassy shoot...,https://nypost.com/2025/08/07/us-news/gunman-c...
18,37,2025-08-07-18-28-09 +0000,bbc,Netanyahu says Israel intends to take complete...,https://www.bbc.com/news/articles/c8ryekj1m1do...
204,37,2025-08-07-09-00-15 +0000,wapo,"In tariff spat, Trump wields India-Russia trad...",https://www.washingtonpost.com/world/2025/08/0...
66,33,2025-08-07-16-55-38 +0000,nypost,NYC guys are splurging on marathon dates as lo...,https://nypost.com/2025/08/07/lifestyle/nyc-gu...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
